In [ ]:
import sys
sys.path.append('..')

from misc import load, load_class_labels

dataset_root = "../dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train_info_cleaned.csv', 1, sample_rate)
test_info = load(dataset_root + 'labels/test.csv')
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values
test_image_names = test_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
test_labels = test_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [ ]:
from cnn_classification import build_finetune_network, train_network
from tensorflow import keras
from keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input

save_path = "./models/efficientnet_v2_standard"

In [ ]:
extension = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(251, activation='softmax')])

loss = 'categorical_crossentropy'
optimizer = 'adam'
metrics = ['accuracy', keras.metrics.TopKCategoricalAccuracy(3)]

cut_layer = 'top_dropout'
non_trainable_cut_layer = 'block5e_add'

extended_model = build_finetune_network(EfficientNetV2B0, extension, cut_layer,
                                        optimizer, metrics, loss, non_trainable_cut_layer=non_trainable_cut_layer)

In [ ]:
train_loss, train_accuracy, val_loss, val_accuracy = train_network(extended_model,
                                                                   save_path,
                                                                   train_info,
                                                                   val_info,
                                                                   train_dir,
                                                                   epochs=10,
                                                                   augment=True,
                                                                   strong_augment=False,
                                                                   batch_size=64,
                                                                   preprocess_input=preprocess_input)

In [ ]:
from tensorflow import keras
from cnn_classification import evaluate_model, predict

model = keras.models.load_model(save_path)
predictions, names = predict(model, val_dir, 64, preprocess_input=preprocess_input)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

ordered_indices = np.argsort(names)
predictions = np.array(predictions)
ordered_names = np.argsort(test_image_names)

predictions_logit = np.argmax(predictions[ordered_indices], axis=1)
ordered_y = test_labels[ordered_names]


cm = confusion_matrix(ordered_y, predictions_logit)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

In [ ]:
cm.diagonal().sum() / cm.sum()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ordered_y, predictions_logit, target_names=class_labels))

In [ ]:
def top_k(k = 1):
    ohe = keras.utils.to_categorical(ordered_y, num_classes=251)
    m = keras.metrics.TopKCategoricalAccuracy(k)
    ordered_pred = predictions[ordered_indices]
    m.update_state(ohe, ordered_pred)
    return m.result().numpy()

In [ ]:
top_k(1)

In [ ]:
top_k(3)